# OANDA Historical Data Retrieval

This notebook demonstrates how to retrieve historical forex data from OANDA's API.

## Features
- Connect to OANDA practice or live account
- Retrieve historical candle data for any instrument
- Support multiple timeframes (granularities)
- Save data to CSV files
- Visualize price data

## Prerequisites
- OANDA account (practice or live)
- API credentials configured in `config/oanda_config.ini`
- Dependencies installed: `pip install -r requirements.txt`

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

from src.oanda_client import OandaClient
from src.data_retriever import HistoricalDataRetriever
from src.data_storage import DataStorage
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ Imports successful")

## 2. Initialize OANDA Client

Choose your environment (practice or live) and initialize the client.

In [ ]:
# Choose environment: 'practice' or 'live'
ENVIRONMENT = 'practice'

# Initialize client
client = OandaClient(environment=ENVIRONMENT)
retriever = HistoricalDataRetriever(client)
storage = DataStorage()

print(f"✓ Client initialized for {ENVIRONMENT} environment")

## 3. Validate Connection

Verify that your credentials work and you can connect to OANDA.

In [ ]:
# Validate connection
try:
    client.validate_connection()
    print("✓ Connection validated successfully")

    # Get account info
    account_info = client.get_account_info()
    print(f"\nAccount Details:")
    print(f"  Currency: {account_info['currency']}")
    print(f"  Balance: {account_info['balance']}")
    print(f"  NAV: {account_info['NAV']}")
except Exception as e:
    print(f"✗ Connection failed: {str(e)}")
    print("\nPlease check your config/oanda_config.ini file")

## 4. Available Instruments

Get a list of all tradeable instruments available in your account.

In [ ]:
# Get list of available instruments
instruments = client.get_instruments()
print(f"Total available instruments: {len(instruments)}")
print(f"\nFirst 20 instruments:")
for i, instrument in enumerate(instruments[:20], 1):
    print(f"  {i}. {instrument}")

# Filter major forex pairs
major_pairs = [i for i in instruments if any(x in i for x in ['EUR_', 'GBP_', 'USD_', 'JPY_'])]
print(f"\nMajor forex pairs ({len(major_pairs)}):")
print(major_pairs[:15])

## 5. Single Instrument Data Retrieval

Retrieve historical data for a single instrument and timeframe.

In [ ]:
# Specify parameters
INSTRUMENT = 'EUR_USD'
GRANULARITY = 'H1'  # Hourly candles
FROM_DATE = '2025-01-01'
TO_DATE = '2025-12-31'

print(f"Fetching {INSTRUMENT} data...")
print(f"  Granularity: {GRANULARITY}")
print(f"  From: {FROM_DATE}")
print(f"  To: {TO_DATE}")
print("\nThis may take a moment...\n")

# Fetch data
df = retriever.fetch_historical_data(
    instrument=INSTRUMENT,
    granularity=GRANULARITY,
    from_date=FROM_DATE,
    to_date=TO_DATE
)

# Display summary
print(f"\n✓ Retrieved {len(df)} candles")
print(f"  Date range: {df['time'].min()} to {df['time'].max()}")
print(f"  Price range: {df['low'].min():.5f} to {df['high'].max():.5f}")

### View Data Sample

In [ ]:
print("First 5 rows:")
display(df.head())

print("\nLast 5 rows:")
display(df.tail())

print("\nData Info:")
print(df.info())

## 6. Save Data to CSV

Save the retrieved data to a CSV file for later use.

In [ ]:
# Save to CSV
file_path = storage.save_to_csv(
    df=df,
    instrument=INSTRUMENT,
    granularity=GRANULARITY,
    from_date=FROM_DATE,
    to_date=TO_DATE
)

print(f"✓ Data saved to: {file_path}")
print(f"  File size: {len(df)} records")

## 7. Visualize Price Data

Create basic visualizations of the price data.

In [ ]:
# Price chart
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Close price
ax1.plot(df['time'], df['close'], label='Close Price', linewidth=1)
ax1.set_title(f'{INSTRUMENT} - {GRANULARITY} Close Prices', fontsize=14, fontweight='bold')
ax1.set_ylabel('Price', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Volume
ax2.bar(df['time'], df['volume'], alpha=0.5, color='steelblue')
ax2.set_title(f'{INSTRUMENT} - {GRANULARITY} Volume', fontsize=14, fontweight='bold')
ax2.set_xlabel('Time', fontsize=12)
ax2.set_ylabel('Volume', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPrice Statistics:")
print(df[['open', 'high', 'low', 'close']].describe())

## 8. Multiple Instruments Retrieval

Retrieve data for multiple instruments at once.

In [ ]:
# Fetch multiple instruments
INSTRUMENTS = ['EUR_USD', 'GBP_USD', 'USD_JPY']
GRANULARITY = 'H4'  # 4-hour candles
FROM_DATE = '2024-01-01'

print(f"Fetching data for {len(INSTRUMENTS)} instruments...")
print(f"  Instruments: {', '.join(INSTRUMENTS)}")
print(f"  Granularity: {GRANULARITY}")
print(f"  From: {FROM_DATE}")
print("\nThis will take a few moments...\n")

data_dict = retriever.fetch_multiple_instruments(
    instruments=INSTRUMENTS,
    granularity=GRANULARITY,
    from_date=FROM_DATE
)

print("\n✓ Data retrieved for all instruments")
print("\nSummary:")
for instrument, df in data_dict.items():
    print(f"  {instrument}: {len(df)} candles")

### Save All Data

In [ ]:
# Save all instruments
to_date = datetime.now().strftime('%Y-%m-%d')

for instrument, df in data_dict.items():
    file_path = storage.save_to_csv(
        df=df,
        instrument=instrument,
        granularity=GRANULARITY,
        from_date=FROM_DATE,
        to_date=to_date
    )
    print(f"✓ {instrument}: Saved {len(df)} candles to {file_path}")

## 9. Load Existing Data

Load previously saved CSV files.

In [ ]:
# Load previously saved data
loaded_df = storage.load_from_csv(file_path)

print(f"✓ Loaded {len(loaded_df)} candles from {file_path}")
print(f"  Date range: {loaded_df['time'].min()} to {loaded_df['time'].max()}")

display(loaded_df.head())

## 10. List All Available Data

View all CSV files that have been saved.

In [ ]:
# List all available data files
available_data = storage.list_available_data()

if not available_data.empty:
    print(f"Total files: {len(available_data)}")
    print("\nAvailable data files:")
    display(available_data)
else:
    print("No data files found yet. Run the retrieval cells above to fetch data.")

## 11. Bulk Download Example

Download multiple instruments and granularities in one go.

In [ ]:
# Configuration for bulk download
configs = [
    {'instrument': 'EUR_USD', 'granularity': 'M15', 'from': '2024-01-01'},
    {'instrument': 'EUR_USD', 'granularity': 'H1', 'from': '2024-01-01'},
    {'instrument': 'EUR_USD', 'granularity': 'H4', 'from': '2023-01-01'},
    {'instrument': 'EUR_USD', 'granularity': 'D', 'from': '2020-01-01'},
    {'instrument': 'GBP_USD', 'granularity': 'H1', 'from': '2024-01-01'},
    {'instrument': 'USD_JPY', 'granularity': 'H1', 'from': '2024-01-01'},
]

print(f"Starting bulk download of {len(configs)} datasets...\n")

results = []
for i, config in enumerate(configs, 1):
    print(f"[{i}/{len(configs)}] Fetching {config['instrument']} {config['granularity']}...")

    try:
        # Fetch data
        df = retriever.fetch_historical_data(
            instrument=config['instrument'],
            granularity=config['granularity'],
            from_date=config['from']
        )

        # Save to CSV
        file_path = storage.save_to_csv(
            df=df,
            instrument=config['instrument'],
            granularity=config['granularity'],
            from_date=config['from'],
            to_date=datetime.now().strftime('%Y-%m-%d')
        )

        results.append({
            'instrument': config['instrument'],
            'granularity': config['granularity'],
            'records': len(df),
            'status': '✓ Success'
        })

        print(f"  ✓ Saved {len(df)} candles\n")

    except Exception as e:
        print(f"  ✗ Error: {str(e)}\n")
        results.append({
            'instrument': config['instrument'],
            'granularity': config['granularity'],
            'records': 0,
            'status': f'✗ Failed: {str(e)}'
        })

print("\n" + "="*60)
print("BULK DOWNLOAD COMPLETE")
print("="*60)
results_df = pd.DataFrame(results)
display(results_df)

## 12. Available Granularities Reference

List of all supported granularities:

In [ ]:
print("Supported Granularities:")
print("\nSeconds:")
print("  S5, S10, S15, S30")
print("\nMinutes:")
print("  M1, M2, M4, M5, M10, M15, M30")
print("\nHours:")
print("  H1, H2, H3, H4, H6, H8, H12")
print("\nDaily/Weekly/Monthly:")
print("  D (Daily), W (Weekly), M (Monthly)")

## Next Steps

Now that you have historical data, you can:
1. Build trading strategies
2. Perform backtesting
3. Calculate technical indicators
4. Analyze market patterns
5. Develop machine learning models

All your data is saved in the `data/historical/` directory, organized by instrument.